# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [292]:
# Use this cell to write your code for Task 1
library(readr)
library(dplyr)
library(stringr)
missing_city <- read_csv("house_sales.csv", show_col_types = FALSE) %>%	
						pull(city)

missing_city <- length(missing_city[missing_city == '--'])

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [293]:
# Use this cell to write your code for Task 2
data <- read_csv("house_sales.csv", show_col_types = FALSE)
data$city <- ifelse(data$city == '--', 'Unknown', data$city)
#sum(is.na(data$sale_price))
#sum(is.nan(data$sale_price))
#sum(is.null(data$sale_price))
#class(data$sale_price)
bool <- is.na(data$months_listed)
data$months_listed[bool] <- round(mean(data$months_listed, na.rm = TRUE), digits = 1)
data$house_type <- data$house_type %>% recode(Det. = 'Detached', Semi = 'Semi-detached', Terr. = 'Terraced')
data$area <- str_remove_all(data$area, pattern = ' sq.m.') %>% as.numeric()

clean_data <- data

clean_data <- as.data.frame(clean_data)

# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [294]:
# Use this cell to write your code for Task 3
data <- read_csv("house_sales.csv", show_col_types = FALSE)
data$city <- ifelse(data$city == '--', 'Unknown', data$city)
bool <- is.na(data$months_listed)
data$months_listed[bool] <- round(mean(data$months_listed, na.rm = TRUE), digits = 1)
data$house_type <- data$house_type %>% recode(Det. = 'Detached', Semi = 'Semi-detached', Terr. = 'Terraced')
data$area <- str_remove_all(data$area, pattern = ' sq.m.') %>% as.numeric()
price_by_rooms <- data

price_by_rooms <- price_by_rooms %>% 
						group_by(bedrooms) %>%
						summarise(avg_price = round(mean(sale_price), digits = 1),
								  var_price = round(var(sale_price), digits = 1))

price_by_rooms <- as.data.frame(price_by_rooms)

# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [295]:
# Use this cell to write your code for Task 4
train_data <- read_csv("train.csv", show_col_types = FALSE)
#names(train_data)
model <- lm(sale_price ~ city + sale_date + months_listed + bedrooms + house_type + area, data = train_data)
validation_data <- read_csv("validation.csv", show_col_types = FALSE)
prediction <- predict(model, newdata = validation_data)

base_result <- data.frame(house_id = validation_data$house_id,
						  price = prediction)

# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [296]:
# Use this cell to write your code for Task 5
library(xgboost)
train_data <- read_csv("train.csv", show_col_types = FALSE)
#names(train_data)
X <- model.matrix( ~ city + house_type + area, data = train_data)
y = train_data$sale_price
model_2 <- xgboost(data = X, label = y, nrounds = 100, objective = "reg:squarederror", , verbose = 0)
validation_data <- read_csv("validation.csv", show_col_types = FALSE)
preds_xg <- predict(model_2, model.matrix( ~ city + house_type + area, data = validation_data))


compare_result <- data.frame(house_id = validation_data$house_id,
						  price = preds_xg)